# STRUCTURE-BASED VIRTUAL SCREENING (SBVS)
---

**Author** : Natalia García Sánchez - Masters in Computational Biology

**Date** : 02/01/2023


**Description** : *Project for the **Computational Structure Biology for Drug Lead Discovery course***.
    
Group of preprocessing programs and pipelines to implement SBVS docking against **JAK1**, *a target present in a pathway and implicated on the disease progression of Rheumathoid Arthritis*, with `autodock-vina`, over a candidate database of 60 compounds. The former candidate ligand database was obtained through prior Pharmacophore-based Ligand-Based Virtual Screening(LBVS)


- *Dependencies : Linux OS, conda environment administered python packages smina, openbabel, rdkit, fpocket, pandas, seaborn, autodock-vina, mgltools, biopython*

- Binaries for execution of the pipeline such as LeDock and CB-Dock

- **Note on prior steps**

A prior LBVS pipeline needs to be implemented to get the final 60 candidate ligands and the visualization of their ADMET properties can be found in __

**Library import**

In [ ]:
#import packages
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from openbabel import pybel

#### INDEX OF CONTENTS
---

**1. Download PDB structure**

**2. Protein sanitization**

**3. Ligand Sanitization - Ligand battery prep**

**4. Docking**


- *Cavity detection*

    4.1. Manual Approximation
    
    4.2. CB-Dock
    
    4.3. fpocket
    
    
    
- *Target .pdbqt generation*  


- *Multiligand docking with SMINA*

Done with 

    JAK1 - Target for lead discovery
    JAK2
    JAK3

**5. Data postprocessing**

## 1. Download PDB structure

There was a resolved structure for the target at hand, so no protein structure models of the target had to be generated. The structures for the JH1 domain with inhibitors were downloaded from https://www.rcsb.org/structure/3EYG RCSB PDB server. In chimera, both the structure for the domain with and without ligand was downloaded in `3eyg.pdb` and `3eyg_noligand.pdb`. The resolution of the XRAY - Diffraction model was of 1.9 Armstrong, higher than average. 

## 2. Protein Sanitization

In order to "sanitize" or clean the protein from undesireable ligands the LePhar molecular docking tool **LePro** will be implemented. LePro is designed to automatically add hydrogen atoms to proteins and/or nucleic acids by explicitely considering the protonation state of histidine. All crystal water, ions, small ligands and cofactors except HEM are also removed. 

LePro software download: http://www.lephar.com/software.htm 

In [ ]:
! ./lepro_linux_x86

************************************************************
*      LePro                                               *
*            Add hydrogen atoms to a protein &             *
*            write the input file for LeDock               *
*      Copyright (C) 2013-21 Hongtao Zhao, PhD             *
*      Email: htzhaovv@gmail.com                           *
************************************************************
----------Usage:                                                                       
          lepro [PDB file] [-rot || -metal || -p]                                        
          -rot  [[chain] resid] align principal axes of the binding site with Cartesian
          -metal keep ZN/MN/CA/MG                                                      
          -metal -p redistribute metal charge to protein                               



After performing `./lepro_linux_x86 3eyg_noligands.pdb` we will have a "clean" protein pdb file automatically saved as *`pro.pdb`* and a configuration file that can be used with LeDock (just another method). 

---
## 3. Ligand Sanitization - Ligand battery prep

The protein now has a clean structure and H added to it, so it is time to think about the ligands we want to dock onto our protein. Once a list of putative ligands/interactors has been created they need to be prepared for the docking protocol. This is a simple step that can be done using two Python packages *rdkit* and *openbabel*. 

In this case the ligands come from a previous LBVS protocol based on pharmacophore selection of candidate compounds with a simmilar fingerprint to tofacitinib. The candidate ligands have also been filtered out if not compliant with a set of ADMET properties described by the Lipinski rule. The ligand file is called `PharmacophoreMolForDocking.sdf`

---


In this way, the following steps will be followed to create the `.mol2` file that will be used by the docking program as a candidate library:

**1. Load the sdf file recently created to a Pandas Dataframe using Rdkit Pandas Tools**


In [ ]:
MoleculeDatabase = PandasTools.LoadSDF('PharmacophoreMolForDocking.sdf', embedProps=True, molColName=None, smilesName='SMILES', removeHs=False)


In [ ]:
MoleculeDatabase.head()

,rmsd,ID,SMILES
0,0.794227779,CHEMBL4638316,CN(C[C@@H]1CCCN1c1ncnc2[nH]ccc12)C(=O)CC#N
1,0.847083986,CHEMBL3689522,C[C@@H]1CN(c2ncnc3[nH]ccc23)C[C@@]2(CCN2C(=O)C...
2,0.0967325717,CHEMBL4062946,C=CC(=O)N1CCC[C@@H](Nc2ncnc3[nH]ccc23)C1
3,0.0967181772,CHEMBL4062806,C=CC(=O)N1C[C@H](C)C[C@@H](Nc2ncnc3[nH]ccc23)C1
4,0.0940026864,CHEMBL4085457,C=CC(=O)N1C[C@H](Nc2ncnc3[nH]ccc23)CC[C@@H]1C


Now that the SDF file is loaded, the following steps will be performed in the next cell


**2. Save SMILES and Library Id into SMILES and ChemblID variables**


**3. Read molecules from SMILES and add a title to each one that can be traced back to the original metabolite database for data postprocessing.**


**4. Create 3D coordinates**: make3D adds H (we already have them, but there is no conflict) and performs a quick local optimization  with 50 steps and the MMFF94 forcefield. We want to further refine this optimization, hence why we use localopt with 500 steps.

---

#### Ligand preparation for SBVS

We will need to generate a miltimolecule `.mol2` file with candidate ligands with RMSD<0.3 Armstrong to pharmacophore candidate obtained in prior LBVS process.

In [ ]:
Final_MolDB = MoleculeDatabase[MoleculeDatabase.rmsd.astype(float)<0.3]
print("no of ligands for docking" , Final_MolDB.shape[0])

SMILES = Final_MolDB["SMILES"].tolist()
ChEMBLIDS = Final_MolDB["ID"].tolist()


no of ligands for docking 25


In [ ]:
out=pybel.Outputfile(filename='candidate_ligands_docking.mol2',format='mol2',overwrite=True) #open file to read and write

for smi in (SMILES):
        mol=pybel.readstring(string=smi,format='smiles') #read mol from SMILE

for index, chemid in enumerate(ChEMBLIDS):
        mol.title='mol_'+str(chemid)+'_'+str(index) #title = mol_libraryID_index

        mol.make3D('mmff94s') #write mols 3D coordinates
        mol.localopt(forcefield='mmff94s', steps=500) #Locally optimize the coordinates

        out.write(mol)
out.close()

no of ligands for docking 25


#### Ligand preparation for docking-cavity detection prediction.

We will also want to predict the JAK1 drug target binding cavity coordinates, size and volume. As we will see in the following section, this can be done with a program called CB-Dock. However, the program will need a file with the .pdb information of the protein target, and a file with one or more ligand coordinates in a `.pdb` or`.mol2`

**To ensure that the information for cavity prediction is reliable**, we will stablish a **strict rmsd cutoff**  under **0.095**, associated to the distance from the pharmacophore against the candidate ligands. This will ensure that the prediction is based on ligands more similar to our pharmacophore, and will also help with the small ligand file size limit required to run CB-Dock.

In [ ]:
Pocket_MolDB = MoleculeDatabase[MoleculeDatabase.rmsd.astype(float)<0.095]

In [ ]:
print("no of ligands for pocket detection" , Pocket_MolDB.shape[0])

no of ligands for pocket detection 2


In [ ]:
SMILES_p = Pocket_MolDB["SMILES"].tolist()
ChEMBLIDS_p = Pocket_MolDB["ID"].tolist()

In [ ]:
out=pybel.Outputfile(filename='candidate_ligands_pocket.mol2',format='mol2',overwrite=True) #open file to read and write

for smi in (SMILES_p):
        mol=pybel.readstring(string=smi,format='smiles') #read mol from SMILE

for index, chemid in enumerate(ChEMBLIDS_p):
        mol.title='mol_'+str(chemid)+'_'+str(index) #title = mol_libraryID_index

        mol.make3D('mmff94s') #write mols 3D coordinates
        mol.localopt(forcefield='mmff94s', steps=500) #Locally optimize the coordinates

        out.write(mol)
out.close()

---
## 4. Docking 

### 4.1. Docking coordinate calculation

To have an appoximate insight of the coordinates to which the ligands will be docked to our target JAK1, we used the following approaches.

1. Manual approximation : Approximate description of binding sight and approximate center coordinate of tofacitinib inhibitor molecule in JAK1
2. fpocket: very easy to use, can be installed through conda
3. CB-Dock

#### 4.1.1. Manual approximation

You will need the chimera software (https://www.cgl.ucsf.edu/chimera/download.html) for this step.
First of all, we will eliminate the H2O residues with *Select>Residue>H2O* option available at the Main Toolbar. After sellection, we will run the command *dele sel*. Now we do the same selecting the *Tofacitinib* molecule MIT. To check collindant residues most likely to participate in an interaction with the compound with distance d>0.4A from it, we will use the *Select>Zone* option and specify the threshold from the previously selected MIT.

Prior analysis in Chimera software shows that ligands under 0.3 Armstrong from the initial inhibitor compound (taken as a reference for ligand search and analysis - Tofacitinib) has JAK1 residues:
- Gly884
- Glu957
- Leu 953

In addition, Tofacitinib most noticeably centric molecule to the Target protein visualized by chimera, C1 from MIT (tofacitinib PDB ligand accession) has coordinates $(x, y, z)=(11.400, 14.076, -15.349)$

#### 4.1.2. CB-Dock

CB-Dock is a protein-ligand docking method which automatically identifies the binding sites, calculates the center and size, customizes the docking box size according to the query ligands and performs the molecular docking with AutoDock Vina. It needs the autodock-vina and mgltools packages

```
    conda install -c bioconda autodock-vina
    conda install -c bioconda mgltools

    ##other set up config

    ##run CB-dock
    perl ./prog/AutoBlindDock.pl [pro.pdb] [candidate_ligands_pocket.mol2] [5] [output_dir]
```
We will use this tool to have a more precise insight on the drug binding cavity center coordinates and sizes for the following docking.
**This is the result of center coordinates and sizes of cavities of JAK1** given in a `protein_CB_dock_coords.txt` file
```
Cavities	center_x	center_y	center_z	size_x	size_y	size_z	cavity_size
1	11	13	-18	15	14	16	630
2	-5	6	-17	8	11	18	319
3	1	21	2	8	14	6	178
4	9	8	-1	12	5	4	168
5	29	1	-24	6	8	10	113
```

#### 4.1.3. fpocket

We will also try this cavity detection algoritythm to validate if the main cavity features predicted are robust or consistent with the results for the manual chimera gathering of coordinates. Although it does not give us the calculated coordinates and box-sizes because it is not a docking-cavity searcher, it does give the coordinates of every atom (in each residue) that compose the pocket/cavity, and   aspects relating to druggability and other features like hidrophobicity, volume, flexibility, charge, Solvent Accesible Surface,... predicted in the pockets. We will check if the Volume predicted matches some of those predicted by CB-Dock

Run this line on the terminal after installation

```
    fpocket pro.pdb
```
After running this line, pdb files with protein residues predicted to be present in each of the 23 predicted pockets are gathered in a file called  `pro_out`. Also, a file in this folder called `pro_info.txt` provides a summary of features regarding a *prediction score, Total Volume in Armstrong³, Druggability scores, Hidrophobicity and B-Factor, etc*.

**Output:** the second pocket predicted with a score of 0.156 and a druggability score of 0.049, could correspond with some variations to a simmilar cavity C1 of that predicted by CB-Dock. This conclusions are made on the basis of several evidences:

- Volume predicted was of 741.38 Armstrong, simmilar to that of the first cavity in CB-Dock (603A)
- CB-Dock first cavity has center coordinates simmilar to those determined in chimera
- PDB files retrieved in fpocket retrieved residues implicated in pockets close to those determined manually in chimera.

#### 4.1.4. Receptor prep - PDBQT file generation 

The receptor file should have information about charge. To calculate this information and convert the previous cleaned `pro.pdb` file into the receptor of the docking file `receptor.pdb`, one should use the `prepare_receptor4.py` python2 script available in the CB-Dock directory `CB-Dock/prog/ADT_scripts/`. It should be done by executing in the terminal the following command:

    ./prepare_receptor4.py -v -r /path_to_file/pro.pdb -o /path_to_file/receptor.pdbqt


### 4.2. Docking using Smina

Smina can be installed in a conda environment through the bioconda channel and just needs to be called adding the required and desired attributes. 

```
#!/bin/bash

cat protein_CB_dock_coords.txt | head -n2 | while IFS=$' ' read line;
 
do 
x=$(echo $line | cut -d" " -f2)
y=$(echo $line | cut -d" " -f3)
z=$(echo $line | cut -d" " -f4)
j=$(echo $line | cut -d" " -f5)
k=$(echo $line | cut -d" " -f6)
l=$(echo $line | cut -d" " -f7)

smina -r receptor.pdbqt -l candidate_ligands_docking.mol2 -o protein_smina.sdf --center_x "$x" --center_y "$y" --center_z "$z" --size_x "$j" --size_y "$k" --size_z "$l" --exhaustiveness 8 --num_modes 1 --seed 7683; 

done


```

As an output smina produces an sdf file containing the 3D coordinates of all docked molecules. 

---



## 5. Data Postprocessing

This section will deal with the merging of the smina output file and the initial ligand candidate lists by the custom pybel ID given in **Section 2 - Ligand prep**.

It will also calculate $K_d$ dissociation factor values based on the following formula : 

$$K_d = \exp(\frac{\Delta G}{RT})$$


Where RT takes the value of $0.00198 \frac{kcal}{mol·K} · 300 K = 0.592 \frac{kcal}{mol}$

In [ ]:
numbers = list(range(0,5)) #number of metabolites
numbers_ = [] #create name "mol_" + "number" from 0 to # of metabolites

for index, lib in enumerate(ChEMBLIDS):
    name='mol_'+str(lib)+'_'+str(index) #molecule descriptor same as above
    numbers_.append(name)

all_df = pd.DataFrame(list(zip(SMILES, ChEMBLIDS, numbers_))) #create a dataframe with all available info on each metabolite
all_df =all_df.rename(columns={1:'ID'})

In [ ]:
import numpy as np
sminaJAK1 = PandasTools.LoadSDF('JAK1_smina.sdf', embedProps=True, molColName=None, removeHs=False)
sminaJAK1['Kd(JAK1) nM'] = np.exp(sminaJAK1.minimizedAffinity.astype(float)/0.592)*(10**9)

ID_reformatted = []
ID_list = sminaJAK1.ID.to_list()

for ID_Chembl in ID_list:
    ID_reformatted.append(ID_Chembl.split('_')[2])
sminaJAK1['ID'] = ID_reformatted
sminaJAK1


,minimizedAffinity,ID,Kd(JAK1) nM
0,-9.20881,CHEMBL4062946,175.535893
1,-9.35053,CHEMBL4062806,138.165418
2,-9.59700,CHEMBL4085457,91.114521
3,-9.66870,CHEMBL4083165,80.721287
4,-9.46661,CHEMBL4074183,113.564456
5,-9.67994,CHEMBL4069969,79.203131
6,-9.45590,CHEMBL4095629,115.637672
7,-9.49955,CHEMBL3715588,107.418098
8,-9.67417,CHEMBL3719354,79.978869
9,-9.68269,CHEMBL3715974,78.836065


In [ ]:
sminaJAK1 = sminaJAK1[['ID','Kd(JAK1) nM']]

In [ ]:
merge = pd.merge(sminaJAK1, all_df, on ='ID')[['ID', 0, 'Kd(JAK1) nM']]
merge

,ID,0,Kd(JAK1) nM
0,CHEMBL4062946,C=CC(=O)N1CCC[C@@H](Nc2ncnc3[nH]ccc23)C1,175.535893
1,CHEMBL4062806,C=CC(=O)N1C[C@H](C)C[C@@H](Nc2ncnc3[nH]ccc23)C1,138.165418
2,CHEMBL4085457,C=CC(=O)N1C[C@H](Nc2ncnc3[nH]ccc23)CC[C@@H]1C,91.114521
3,CHEMBL4083165,C=CC(=O)N1CC[C@@H](C)[C@H](Nc2ncnc3[nH]ccc23)C1,80.721287
4,CHEMBL4074183,C=CC(=O)N1C[C@@H](C)C[C@H](Nc2ncnc3[nH]ccc23)C1,113.564456
5,CHEMBL4069969,C=CC(=O)N1CC[C@H](C)[C@H](Nc2ncnc3[nH]ccc23)C1,79.203131
6,CHEMBL4095629,C/C=C/C(=O)N1CCC[C@@H](Nc2ncnc3[nH]ccc23)C1,115.637672
7,CHEMBL3715588,NC(=O)c1ccc2[nH]c3ncnc(N4CCCCC4)c3c2c1,107.418098
8,CHEMBL3719354,NC(=O)c1ccc2[nH]c3ncnc(N4CCCCC4)c3c2c1,79.978869
9,CHEMBL3715974,NC(=O)c1ccc2[nH]c3ncnc(N4CCCCCC4)c3c2c1,78.836065


In [ ]:
#merge = merge.astype({'minimizedAffinity': 'float'})
merge_sorted = merge.sort_values('Kd(JAK1) nM')
merge_sorted

,ID,0,Kd(JAK1) nM
15,CHEMBL3715062,CCCN(CCC)c1ncnc2[nH]c3ccc(C(N)=O)cc3c12,77.562711
23,CHEMBL78377,Cc1[nH]c2nc(-c3ccccc3)nc(NCCCN3CCCC3=O)c2c1C,78.422995
21,CHEMBL4087971,CN1CCN([C@H]2CC[C@H](Nc3ncnc4ccc(C#N)cc34)CC2)CC1,78.746892
9,CHEMBL3715974,NC(=O)c1ccc2[nH]c3ncnc(N4CCCCCC4)c3c2c1,78.836065
5,CHEMBL4069969,C=CC(=O)N1CC[C@H](C)[C@H](Nc2ncnc3[nH]ccc23)C1,79.203131
8,CHEMBL3719354,NC(=O)c1ccc2[nH]c3ncnc(N4CCCCC4)c3c2c1,79.978869
16,CHEMBL4545883,C[C@H]1CCCCN1CC(C)(C)Nc1nc(-c2ccncc2)nc2cnccc12,80.691295
3,CHEMBL4083165,C=CC(=O)N1CC[C@@H](C)[C@H](Nc2ncnc3[nH]ccc23)C1,80.721287
10,CHEMBL3716162,CC1CCN(c2ncnc3[nH]c4ccc(C(N)=O)cc4c23)CC1,81.932929
18,CHEMBL4649816,O=C1C[C@@H](Nc2ncnc3nc[nH]c23)CN1Cc1ccccc1,82.683709


## Determining docking characteristics in other members of the JAK family

The same procedure we went through in the last stages (Downloading PDB, Receptor prepping, Cavity detection, Smina docking analysis) will be applied to JAK2 and JAK3, other members of the JAK Janus Kinase family. This is done to if our candidate ligands will show affinity to these other members affecting othe pro-inflammatory recognition and signal transduction pathways of the JAK/STAT pathway, and therefore leading to unwanted side effects and higher dose-toxicity. We want our candidates to have leads that point out to less dose-toxicity than that of tofacitinib

The following section will give brief descriptions of some of the decisions and results in several stages of this pipeline
---
- **Download PDB files**. JAK3 PDB accession file 6DUD was chosen on the basis of its high Resolution of 1.66Â. The same criteria was used to download a structure entry in PDB for JAK2, which was specifically that assigned to the 7F7W entry (Resolution of 1.83Â)
- **Cavity Detection**
* **JAK3**: Manual approximation of a coordinate of its entry ligand ([cyanamide HB4](https://www.rcsb.org/ligand/HB4)) Carbon 4 in chimera pointed out to the coordinates $(x, y, z) = (-1.61, -18.41, -5.87)$. The residues closest to the ligand in the entry were those with enumeration *903, 909 and 912*. Prediction of pockets in fpocket pointed out to a pocket (2nd pocket) containing this residues, with predicted scores and druggability of $0.115$ and $0.155$ respectively, and a volume of $1222 A³$. These two levels of evidence were approximately similar to the fist cavity in CB-Dock, used in Smina and presenting a result of :

```
Cavities	center_x	center_y	center_z	size_x	size_y	size_z	cavity_size
1	-8	13	-10	25	14	17	1736

```

* **JAK2**: Manual approximation of a coordinate of its entry ligand ([subject of investigation 36H](https://www.rcsb.org/ligand/36H)) Carbon 11 in chimera pointed out to the coordinates $(x, y, z) = (22.96,-15.92, 4.79)$. Some residues closest to the ligand in the entry wenumerated from 626 to 633. fpocket pointed out to a pocket (1st pocket) containing this residues, with predicted scores and druggability of $0.343$ and $0.894$ respectively, and a volume of $843 A³$. These two levels of evidence were approximately similar to the fist cavity in CB-Dock, used in Smina and presenting a result of :

```
Cavities	center_x	center_y	center_z	size_x	size_y	size_z	cavity_size
1	26	-8	8	11	19	17	1221
```

We will now load this sdf file onto a pandas dataframe (as I have showed you before) and merge both lists by the library ID that is the comon aspect of both dataframes. Once you have done this, you should sort the docking results by value (most negative to most positive).



save 10 first ligands with highest affinity for docking with JAK2 and JAK3

In [ ]:
ligands_JAK23 = merge_sorted[['ID', 0]][0:1O]
SMILES = ligands_JAK23[0].tolist()
ChEMBLIDS = ligands_JAK23["ID"].tolist()

In [ ]:
out=pybel.Outputfile(filename='candidate_ligands_JAK23.mol2',format='mol2',overwrite=True) #open file to read and write

for smi in (SMILES):
        mol=pybel.readstring(string=smi,format='smiles') #read mol from SMILE

for index, chemid in enumerate(ChEMBLIDS):
        mol.title='mol_'+str(chemid)+'_'+str(index) #title = mol_libraryID_index

        mol.make3D('mmff94s') #write mols 3D coordinates
        mol.localopt(forcefield='mmff94s', steps=500) #Locally optimize the coordinates

        out.write(mol)
out.close()

loading docked files

In [ ]:
import numpy as np
sminaJAK2 = PandasTools.LoadSDF('JAK2_smina.sdf', embedProps=True, molColName=None, removeHs=False)
sminaJAK2['Kd(JAK2) nM'] = np.exp(sminaJAK2.minimizedAffinity.astype(float)/0.592)*(10**9)

ID_reformatted = []
ID_list = sminaJAK2.ID.to_list()

for ID_Chembl in ID_list:
    ID_reformatted.append(ID_Chembl.split('_')[1])
sminaJAK2['ID'] = ID_reformatted
sminaJAK2

,minimizedAffinity,ID,Kd(JAK2) nM
0,-9.73508,CHEMBL3715062,72.159139
1,-9.53213,CHEMBL78377,101.666199
2,-10.11269,CHEMBL4087971,38.130680
3,-10.11269,CHEMBL3715974,38.130680
4,-10.11269,CHEMBL4069969,38.130680
5,-10.11269,CHEMBL3719354,38.130680
6,-10.11269,CHEMBL4545883,38.130680
7,-10.11269,CHEMBL4083165,38.130680
8,-10.11269,CHEMBL3716162,38.130680
9,-10.11269,CHEMBL4649816,38.130680


In [ ]:
import numpy as np
sminaJAK3 = PandasTools.LoadSDF('JAK3_smina.sdf', embedProps=True, molColName=None, removeHs=False)
sminaJAK3['Kd(JAK3) nM'] = np.exp(sminaJAK3.minimizedAffinity.astype(float)/0.592)*(10**9)

ID_reformatted = []
ID_list = sminaJAK3.ID.to_list()

for ID_Chembl in ID_list:
    ID_reformatted.append(ID_Chembl.split('_')[2])
sminaJAK3['ID'] = ID_reformatted
sminaJAK3

,minimizedAffinity,ID,Kd(JAK3) nM
0,-3.94671,CHEMBL3715062,1.272541e+06
1,-3.91401,CHEMBL78377,1.344809e+06
2,-4.11760,CHEMBL4087971,9.534673e+05
3,-4.11760,CHEMBL3715974,9.534673e+05
4,-4.11760,CHEMBL4069969,9.534673e+05
5,-4.11760,CHEMBL3719354,9.534673e+05
6,-4.11760,CHEMBL4545883,9.534673e+05
7,-4.11760,CHEMBL4083165,9.534673e+05
8,-4.11760,CHEMBL3716162,9.534673e+05
9,-4.11760,CHEMBL4649816,9.534673e+05


In [ ]:
sminaJAK2 = sminaJAK2[['ID','Kd(JAK2) nM']]
sminaJAK3 = sminaJAK3[['ID','Kd(JAK3) nM']]

merge2 = pd.merge(sminaJAK2, sminaJAK3, how="inner", on='ID')


In [ ]:
final_merge = pd.merge(merge_sorted, merge2, how="inner", on='ID')
final_merge

,ID,0,Kd(JAK1) nM,Kd(JAK2) nM,Kd(JAK3) nM
0,CHEMBL3715062,CCCN(CCC)c1ncnc2[nH]c3ccc(C(N)=O)cc3c12,77.562711,72.159139,1.272541e+06
1,CHEMBL78377,Cc1[nH]c2nc(-c3ccccc3)nc(NCCCN3CCCC3=O)c2c1C,78.422995,101.666199,1.344809e+06
2,CHEMBL4087971,CN1CCN([C@H]2CC[C@H](Nc3ncnc4ccc(C#N)cc34)CC2)CC1,78.746892,38.130680,9.534673e+05
3,CHEMBL3715974,NC(=O)c1ccc2[nH]c3ncnc(N4CCCCCC4)c3c2c1,78.836065,38.130680,9.534673e+05
4,CHEMBL4069969,C=CC(=O)N1CC[C@H](C)[C@H](Nc2ncnc3[nH]ccc23)C1,79.203131,38.130680,9.534673e+05
5,CHEMBL3719354,NC(=O)c1ccc2[nH]c3ncnc(N4CCCCC4)c3c2c1,79.978869,38.130680,9.534673e+05
6,CHEMBL4545883,C[C@H]1CCCCN1CC(C)(C)Nc1nc(-c2ccncc2)nc2cnccc12,80.691295,38.130680,9.534673e+05
7,CHEMBL4083165,C=CC(=O)N1CC[C@@H](C)[C@H](Nc2ncnc3[nH]ccc23)C1,80.721287,38.130680,9.534673e+05
8,CHEMBL3716162,CC1CCN(c2ncnc3[nH]c4ccc(C(N)=O)cc4c23)CC1,81.932929,38.130680,9.534673e+05
9,CHEMBL4649816,O=C1C[C@@H](Nc2ncnc3nc[nH]c23)CN1Cc1ccccc1,82.683709,38.130680,9.534673e+05


In [ ]:
final_merge.to_csv('final_candidates_inc_dock.csv', index=False)

By performing docking* on the Tofacitinib ligand against 3eig.pdb with no ligand (JAK1 without ligand), JAK2 7f7w with no ligand, and JAK3 6dud with no ligand, the compound with a highest score achieved a minimized affinity of -9,-8.2,-7.9 kcal/mol respectively. The results are slightly and perciebably better for our new curated and screened candidate ligands, as they present more affinity for JAK1 than tofacitinib. However, disparity of affinity between the candidates docking in JAK1 and the docking in JAK2 and JAK3 was not as perciebable as with tofacitinib. As a matter of fact, affinity values for JAK2 were higher in the majority of the ligand candidates, possibly because of higher resemblance between JAK1 and JAK2 targets. The only potential candidate without this is CHEMBL78377, an experimental compound without recorded bioactivity for JAK1.

(*) Docking was performed separately in the chimera software using a Vina executable. For this, models of the ligand and protein were saved in separate pdb files, and clicking on Tools>Structure Editing, the options AddH (adding hydrogens) and AddCharges (adding gasteiger charges) was used. After this, clicking on the Tools>StructureBinding>AutodockVina tool, one could specify the final docking file and executables. For the specification of the docking box and coordinates, the ones from MIT(C1) in the initial complete 3eig model were used, as they were considered a sufficient approximation of a centric coordinate in the drug binding site.